# Perlin Noise Vector Field with Updaters

A circular path in noise space drives a time-dependent 2D vector field, and Manim updaters rotate and recolor a grid of arrows accordingly. See the [Arrow reference](https://docs.manim.community/en/stable/reference/manim.mobject.geometry.arrow.Arrow.html) and [updater docs](https://docs.manim.community/en/stable/reference/manim.mobject.updaters.Updater.html) for API details.


In [1]:

from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Tuple

import numpy as np
from manim import *

# Minimal Perlin sampler so the notebook has no extra runtime dependencies.
def _fade(t: np.ndarray | float) -> np.ndarray | float:
    return 6 * t**5 - 15 * t**4 + 10 * t**3

def _lerp(a: np.ndarray | float, b: np.ndarray | float, t: np.ndarray | float) -> np.ndarray | float:
    return a + t * (b - a)

@dataclass
class PerlinSampler:
    seed: int = 1

    def __post_init__(self) -> None:
        rng = np.random.default_rng(self.seed)
        perm = np.arange(256, dtype=int)
        rng.shuffle(perm)
        self.permutation = np.concatenate([perm, perm])

    def _gradient(self, hash_value: np.ndarray) -> np.ndarray:
        gradients = np.array(
            [
                [1, 1, 0],
                [-1, 1, 0],
                [1, -1, 0],
                [-1, -1, 0],
                [1, 0, 1],
                [-1, 0, 1],
                [1, 0, -1],
                [-1, 0, -1],
                [0, 1, 1],
                [0, -1, 1],
                [0, 1, -1],
                [0, -1, -1],
            ]
        )
        return gradients[hash_value % gradients.shape[0]]

    def __call__(self, x: float, y: float, z: float) -> float:
        x_arr = np.asarray(x, dtype=float)
        y_arr = np.asarray(y, dtype=float)
        z_arr = np.asarray(z, dtype=float)

        xi = np.floor(x_arr).astype(int) & 255
        yi = np.floor(y_arr).astype(int) & 255
        zi = np.floor(z_arr).astype(int) & 255
        xf, yf, zf = x_arr - np.floor(x_arr), y_arr - np.floor(y_arr), z_arr - np.floor(z_arr)
        u, v, w = _fade(xf), _fade(yf), _fade(zf)

        def gradient_at(dx: float, dy: float, dz: float, px: int, py: int, pz: int) -> float:
            hash_value = self.permutation[self.permutation[self.permutation[xi + px] + yi + py] + zi + pz]
            gradient = self._gradient(hash_value)
            return float(gradient[..., 0] * dx + gradient[..., 1] * dy + gradient[..., 2] * dz)

        n000 = gradient_at(float(xf), float(yf), float(zf), 0, 0, 0)
        n100 = gradient_at(float(xf - 1), float(yf), float(zf), 1, 0, 0)
        n010 = gradient_at(float(xf), float(yf - 1), float(zf), 0, 1, 0)
        n110 = gradient_at(float(xf - 1), float(yf - 1), float(zf), 1, 1, 0)
        n001 = gradient_at(float(xf), float(yf), float(zf - 1), 0, 0, 1)
        n101 = gradient_at(float(xf - 1), float(yf), float(zf - 1), 1, 0, 1)
        n011 = gradient_at(float(xf), float(yf - 1), float(zf - 1), 0, 1, 1)
        n111 = gradient_at(float(xf - 1), float(yf - 1), float(zf - 1), 1, 1, 1)

        x1 = _lerp(n000, n100, u)
        x2 = _lerp(n010, n110, u)
        x3 = _lerp(n001, n101, u)
        x4 = _lerp(n011, n111, u)
        y1 = _lerp(x1, x2, v)
        y2 = _lerp(x3, x4, v)
        return float(_lerp(y1, y2, w))


/Users/lucasstarkey/Projects/ForFun/manim-sandbox/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Map grid points into noise space

Each grid anchor is scaled into noise coordinates, offset by a circular trajectory, and sampled twice to derive a vector direction and magnitude. Colors come from a small gradient tied to the instantaneous magnitude.


In [2]:

noise_sampler = PerlinSampler(seed=7)
noise_scale = 0.6
noise_radius = 0.8
noise_speed = 0.9

palette: list[ManimColor] = color_gradient([BLUE_E, BLUE_C, GREEN_B, YELLOW_C, RED_C], 64)

def sample_vector_field(point: np.ndarray, time: float) -> Tuple[np.ndarray, float]:
    px, py = point[:2]
    noise_center = np.array([math.cos(time * noise_speed), math.sin(time * noise_speed), 0.0]) * noise_radius
    domain = np.array([px, py, time * 0.35]) * noise_scale + noise_center
    dx = noise_sampler(*(domain + np.array([0.0, 0.0, 0.5])))
    dy = noise_sampler(*(domain + np.array([2.3, -1.7, -0.25])))
    vector = np.array([dx, dy])
    magnitude = float(np.linalg.norm(vector))
    if magnitude > 1e-6:
        vector = vector / magnitude
    vector *= 0.9 * (0.35 + 0.65 * smooth(np.clip(magnitude, 0, 1)))
    return vector, magnitude

def color_from_magnitude(magnitude: float) -> ManimColor:
    scaled = float(np.clip(magnitude * 0.8 + 0.2, 0, 1))
    index = int(scaled * (len(palette) - 1))
    return palette[index]


## Animate a grid of arrows

An updater advances a shared time tracker, while per-arrow updaters rotate and recolor based on the evolving field. For more on composing scenes, see the [Scene class](https://docs.manim.community/en/stable/reference/manim.scene.scene.Scene.html).


In [3]:

class PerlinNoiseVectorField(Scene):
    def construct(self) -> None:
        time_tracker = ValueTracker(0.0)
        time_tracker.add_updater(lambda mob, dt: mob.increment_value(dt))
        self.add(time_tracker)

        def make_arrow(anchor: np.ndarray) -> Arrow:
            return Arrow(
                anchor,
                anchor + RIGHT * 0.01,
                buff=0,
                max_stroke_width_to_length_ratio=2.5,
                max_tip_length_to_length_ratio=0.35,
                stroke_width=3.0,
            )

        def update_arrow(arrow: Arrow, anchor: np.ndarray) -> None:
            vector, magnitude = sample_vector_field(anchor, time_tracker.get_value())
            end = np.append(vector, 0.0) + anchor
            arrow.put_start_and_end_on(anchor, end)
            arrow.set_color(color_from_magnitude(magnitude))

        xs = np.linspace(-4.5, 4.5, 10)
        ys = np.linspace(-2.5, 2.5, 7)
        anchors = [np.array([x, y, 0.0]) for x in xs for y in ys]
        arrow_list: list[Arrow] = [make_arrow(anchor) for anchor in anchors]
        arrows = VGroup(*arrow_list)

        for arrow, anchor in zip(arrow_list, anchors):
            arrow.add_updater(lambda mob, dt, anchor=anchor: update_arrow(mob, anchor))

        self.play(FadeIn(arrows), run_time=1.5)
        self.wait(8.0)
        for arrow in arrow_list:
            arrow.clear_updaters()
        time_tracker.clear_updaters()
        self.play(FadeOut(arrows), run_time=1.0)
